In [3]:
import h5py
import numpy as np

# %hist -o -g -f ipython_history.md

In [4]:
def get_matlab_data(file_path, is_label: bool):
    """reading a .mat file to extract the images or the labels"""

    

    with h5py.File(file_path, 'r') as f:
        
        print(f"Started reading {file_path} . . . ")
    
        data_keys =  list(f.keys())[1]
        refs = f[data_keys][0][:] # get object references
        num_of_examples = len(refs)
    
        if is_label: # Check if the this is a labels file
            examples_data =  np.array(
                [''.join(map(chr, np.squeeze(f[refs[i]]))) for i in range(num_of_examples)]
                ) 
        else: # If not labels file
            examples_data =  np.array(
                [np.transpose(np.array(f[refs[i]])) for i in range(num_of_examples)]
                ) # Taking all  

        print(f"Completed reading {file_path}")    
    
    return examples_data


In [5]:
# Get features
train_data = get_matlab_data('Data/train_data.mat', is_label=False)
validation_data = get_matlab_data('Data/val_data.mat', is_label=False)
test_data = get_matlab_data('Data/test_data.mat', is_label=False)

Started reading Data/train_data.mat . . . 
Completed reading Data/train_data.mat
Started reading Data/val_data.mat . . . 
Completed reading Data/val_data.mat
Started reading Data/test_data.mat . . . 
Completed reading Data/test_data.mat


In [6]:
print(f"The training data has a shape of {train_data.shape}")
print(f"The validation data has a shape of {validation_data.shape}")
print(f"The test data has a shape of {test_data.shape}")

The training data has a shape of (29894, 128, 128)
The validation data has a shape of (3733, 128, 128)
The test data has a shape of (3733, 128, 128)
